In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import chardet

In [4]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample

In [3]:
file_path = 'IRENA_RenewableEnergy_Statistics_2000-2022.csv'

with open(file_path, 'rb') as f:
    result = chardet.detect(f.read())

df_irena = pd.read_csv(file_path, encoding=result['encoding'])

file_path_1 = 'organised_Gen.csv'

with open(file_path_1, 'rb') as f:
    result = chardet.detect(f.read())

df_us_data = pd.read_csv(file_path_1, encoding=result['encoding'])

file_path_2 = '02 modern-renewable-energy-consumption.csv'

with open(file_path_2, 'rb') as f:
    result = chardet.detect(f.read())

df_world_data = pd.read_csv(file_path_2, encoding=result['encoding'])

In [13]:
df_filtered = df_world_data[(df_world_data['Entity'] != 'World') & (df_world_data['Code'].notna())]
df_filtered['High_Solar'] = (df_filtered['Solar Generation - TWh'] > 5).astype(int)
feature_cols = ['Geo Biomass Other - TWh', 'Wind Generation - TWh', 'Hydro Generation - TWh']
df_model = df_filtered.dropna(subset=feature_cols + ['High_Solar'])

/var/folders/2r/46mxdql52bn78bp6pr2cps340000gn/T/ipykernel_9581/4065852586.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['High_Solar'] = (df_filtered['Solar Generation - TWh'] > 5).astype(int)


In [15]:
df_majority = df_model[df_model.High_Solar == 0]
df_minority = df_model[df_model.High_Solar == 1]
df_minority_upsampled = resample(df_minority, replace=True, n_samples=len(df_majority), random_state=42)
df_upsampled = pd.concat([df_majority, df_minority_upsampled])


X_up = df_upsampled[feature_cols]
y_up = df_upsampled['High_Solar']

scaler = StandardScaler()
X_up_scaled = scaler.fit_transform(X_up)

X_train_up, X_test_up, y_train_up, y_test_up = train_test_split(X_up_scaled, y_up, test_size=0.2, random_state=42)

svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train_up, y_train_up)

y_pred_svm = svm_model.predict(X_test_up)
report_svm = classification_report(y_test_up, y_pred_svm, output_dict=False)

svm_coef = svm_model.coef_
svm_intercept = svm_model.intercept_

print(svm_coef)
print(svm_intercept)

[[ 2.21406965 11.27627041 -2.154302  ]]
[4.14551801]


In [16]:
print(report_svm)

              precision    recall  f1-score   support

           0       0.91      0.97      0.94       795
           1       0.97      0.91      0.94       816

    accuracy                           0.94      1611
   macro avg       0.94      0.94      0.94      1611
weighted avg       0.94      0.94      0.94      1611

